# DECSKS-20: 1D1V Collisionless DC sheath problem

We consider the Vlasov equations

$$\partial_t f_e + v_x\partial_x f_e + \partial_x\phi \partial_{v_x} f_e = 0, \qquad\qquad x\in [0, L], v_x\in [-V, V], t\in [0,T], \qquad L,T,V\in\mathbb{R}$$

$$\partial_t f_i + v_x\partial_x f_i - \frac{1}{\mu}\partial_x\phi \partial_{v_x} f_i = 0 \qquad \qquad x\in [0, L], v_x\in [-V, V], t\in [0,T], \qquad L,V,T\in\mathbb{R}, \quad \mu = m_i / m_e$$

Coupled by Poisson's equation:

$$\partial_x^2\phi = \int dv (f_e(t,x,v_x) - f_i(t,x,v_x))$$

In steady state ($\partial_t\rightarrow 0$), we have 

$$\partial_t f_e + v_x\partial_x f_e + \partial_x\phi \partial_{v_x} f_e = 0$$

# Setup

We set up Guclu's simulation [reported in 2], which was taken from Kolobov [1]. Note that Kolobov normalized in terms of ionic quantities, whereas we normalize in terms of the electron quantities (same as [2]), hence we follow the parameters given in [2].

An absorbing wall which collects charge carriers as they reach it is situated at $x = 0$, the sheath edge is located at $x = L$, where is located a far enough distance away from the wall so that we can safely assert a quasineutrality condition and elect to measure relative to this bulk plasma potential, that is,

<u>Boundary conditions: Potential</u>
<ol>
<li>
$\phi (0) = -\frac{1}{2}\sigma (0, t), \qquad \sigma(x = 0,t) = \int_0^t dt' \int_{v_x < 0} dv_x\, v_x (f_i(t',0,v_x) - f_e(t',0,v_x))$
$${}$$
<li>
$\phi(L) = 0$
</ol>

Correspondingly, we have

<u>Boundary conditions: density</u>
<ol>
<li> At wall, $x = 0$:
$${}$$
<ol>
<li>$f_e(t,x = 0,v_x < 0) = 0$
$${}$$
<li>
$f_i(t, x = 0, v_x < 0) = 0$
</ol>
$${}$$
<li> At bulk plasma side, $x = L$:
$${}$$
<ol>
<li> $f_e(t,x = L, v_x) = \begin{cases}
\frac{1}{\sqrt{2\pi}}\exp\left(\frac{v_x^2}{2}\right) & v_x < 0 \\
& \\
0 & \text{else}
\end{cases}$
$${}$$
<li> $f_i(t,x = L, v_x) = \begin{cases}
\frac{1}{\sqrt{2\pi \tau / \mu}}\exp\left(\frac{(v_x + c_s)^2}{2\tau / \mu}\right) & v_x < 0 \\
& \\
0 & \text{else}
\end{cases}$
</ol>
$${}$$
where $\mu \simeq 3671.5 \, (\mathrm{^2H - Deuterium}), \tau = T_i / T_e = 1 / 30, c_s = 1 / \sqrt{\mu}$, see <a href = "https://github.com/dsirajud/IPython-notebooks/blob/master/DECSKS-14%20--%20Two_species_Vlasov_simulations/DECSKS-14%20--%20Two_species_Vlasov_simulations.ipynb">DECSKS-14</a> for the normalization details.

</ol>

<u>Domain</u>

<ul>
<li>$x\in [0, L], \qquad L = 20$
<li>$v\in [-V, V], \qquad V = 8$
<li>$t\in [0, T], \qquad T = 2000\cdot 2\pi
</ul>

<u>Grid</u>
<ul>
<li>$N_x = 240$
<li>$N_{vx} = 300$
<li>$N_t = 62832$ steps
</ul>

<u>Solvers</u>
<ul>
<li>advection solver $x$: $\mathrm{LTE} = O(\Delta x^8)$ (finite difference based)
<li>advection solver $v_x$: $\mathrm{LTE} = O(\Delta v_x^8)$ (Fourier based)
<li>Poisson solver for $\phi$: $\mathrm{LTE} = O(\Delta x^6)$ (finite difference based)
<li>time splitting: $\mathrm{LTE} = O(\Delta t^3)$ (Strang/LF2 splitting)
</ul>







## Objective 1: code upgrade -- add in functionality to include source terms $f_{e,i}(t, x = \text{const}, v_x)$ for $t \geq 0$

Currently, we only have the means to provide an initial distribution and to witness the evolution of both species. When we wish to approach problems such as these, i.e. steady state, we want to continually source charge populations until all transients vanish. Thus, we require adding on top of the distribution for every $t$ a source $f_{e,i}(t, x = \text{const}, v_x)$. This is not a large change.

Since the source is constant, it is advantageous to store a matrix <code>S_i</code> and <code>S_e</code> which are the sources for ions and electrons, respectively. And, to tack them onto the distribution after each sweep through <code>lib.convect.scheme</code> which is stepped through in a splitting scheme from the orchestrator <code>lib.split.scheme</code> in time substeps. The question becomes, adding the source term at each time sub step seems, natural, but is it misrepresentative of the source?

consider the LF2 split scheme:

<ol>
<li>$\Delta t / 2$ step in $x$
<ol>
<li>$f_{e}(t + \Delta t / 2, x, v_x) = f_{e}(t, x - v_x\Delta t / 2, v_x)$
<li>$f_{i}(t + \Delta t / 2, x, v_x) = f_{i}(t, x - v_x\Delta t / 2, v_x)$
</ol>
$${}$$
<li> $\Delta t$ in $v_x$
<ol> 
<li>$f_{e}(t + \Delta t, x, v_x) = f_{e}(t, x - v_x \Delta t / 2, v_x - E\Delta t)$
<li>$f_{i}(t + \Delta t, x, v_x) = f_{i}(t, x - v_x \Delta t / 2, v_x + E\Delta t / \mu)$
</ol>
$${}$$
<li> $\Delta t / 2$ in $x$
<ol>
<li>$f_{e}(t + \Delta t, x, v_x) = f_{e}\left(t, x - (v_x - E\Delta t)\Delta t / 2, v_x - E\Delta t \right)$
<li> $f_{i}(t + \Delta t, x, v_x) = f_{i}\left(t, x - (v_x + E\Delta t / \mu)\Delta t / 2, v_x + E\Delta t / \mu
\right)$
</ol>
</ol>

The obvious aspect of this stepthrough which can be forgotten is that the time steps are taken in both phase space variables separately. It is not that we take two steps $\Delta t$, but that we would have an inconsistency if we tried, for example adding in the source after each substep:

<u>Naive idea for including source term (inconsistent)</u>
<ol>
<li> $\Delta t / 2$ step in $x$
<ol>
<li>$f_{e}(t + \Delta t / 2, x, v_x) = f_{e}(t, x - v_x\Delta t / 2, v_x)$
<li>$f_{i}(t + \Delta t / 2, x, v_x) = f_{i}(t, x - v_x\Delta t / 2, v_x)$
</ol>
$${}$$
<font color = "blue">Add in contribution from sources $S_e$, $S_i$</font>
$${}$$
<li> $\Delta t$ in $v_x$
<ol> 
<li>$S_e = f_{e}(t + \Delta t, x, v_x) = f_{e}(t, x - v_x \Delta t / 2, v_x - E\Delta t)$
<li>$S_i = f_{i}(t + \Delta t, x, v_x) = f_{i}(t, x - v_x \Delta t / 2, v_x + E\Delta t / \mu)$
</ol>

$${}$$
<font color = "blue">Add in contribution from sources $S_e$, $S_i$</font>
$${}$$
<li> $\Delta t / 2$ in $x$
<ol>
<li>$f_{e}(t + \Delta t, x, v_x) = f_{e}\left(t, x - (v_x - E\Delta t)\Delta t / 2, v_x - E\Delta t \right)$
<li> $f_{i}(t + \Delta t, x, v_x) = f_{i}\left(t, x - (v_x + E\Delta t / \mu)\Delta t / 2, v_x + E\Delta t / \mu
\right)$
</ol>
$${}$$
<font color = "blue">Add in contribution from sources $S_e$, $S_i$</font>
$${}$$
</ol>

We notice easily that we have inadvertently added in the source terms not three times, but twice the amount as we intended (i.e. we intended to add in the source once per $\Delta t$). It is not that it is unphysical for us to add in this amount, but if we do so then we are being unclear in our problem setup of the strength of our source (inconsistent).

Since we know the fraction of the time step taken (i.e. <code>frac*t.width</code> = time substep), we may add in a fraction of the source for one (either step "A" or "B"), but not both so that in the end we do add the total distribution, and no extra. That is, in our implementation, we actually step through this as (where the fractional time step at each step is $\text{frac}_s$ such that $\sum_s \text{frac}_s = 1$):
$${}$$
for each stage $s$ in split scheme inside a single full time step $\Delta t$:
<ul>
<li> If advecting $x$
<ul>
<li>$f_{e}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{e}(t, x - v_x\cdot\text{frac}_s\cdot \Delta t, v_x)$
<li>$f_{i}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{i}(t, x - v_x\cdot\text{frac}_s\cdot \Delta t, v_x)$
</ul>
$${}$$
<li> If advecting $v_x$
<ul> 
<li>$f_{e}(t + \text{frac}_s\cdot\Delta t, x, v_x) = f_{e}(t, x, v_x - E\cdot \text{frac}_s\cdot \Delta t)$
<li>$f_{i}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{i}(t, x, v_x + E\cdot \text{frac}_s\cdot \Delta t / \mu)$
</ul>
$${}$$
<li> Repeat over all $s$
</ul>

Thus, we could add in the source as in the following if desired, which is what we elect to use:

<table style="width:70%">
<tr><td>Source implementation in a split scheme</td></tr>
<tr><td>
for each stage $s$ in split scheme inside a single full time step $\Delta t$:
<ul>
<li> If advecting $x$
<ul>
<li>$f_{e}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{e}(t, x - v_x\cdot\text{frac}_s\cdot \Delta t, v_x)$
<li>$f_{i}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{i}(t, x - v_x\cdot\text{frac}_s\cdot \Delta t, v_x)$
</ul>
$${}$$
<font color = "blue">Add in <i>fractional</i> contribution sources $\text{frac}\cdot S_e$, $\text{frac}\cdot S_i$</font>:
<ul>
<li>$f_{e}(t, x, v_x) +\hspace{-.3em}= \text{frac}_s\cdot S_e$
<li>$f_{i}(t, x, v_x) +\hspace{-.3em}= \text{frac}_s\cdot S_i$
</ul>
$${}$$
<li> If advecting $v_x$
<ul> 
<li>$f_{e}(t + \text{frac}_s\cdot\Delta t, x, v_x) = f_{e}(t, x, v_x - E\cdot \text{frac}_s\cdot \Delta t)$
<li>$f_{i}(t + \text{frac}_s\cdot \Delta t, x, v_x) = f_{i}(t, x, v_x + E\cdot \text{frac}_s\cdot \Delta t / \mu)$
</ul>
</ul>
</td></tr></table>

So, that $\sum_s \text{frac}_s\cdot S_{e,i} = 1\cdot S_{e,i}$.

This means of implementing a source term is workable. The alternative to this is to add in the contribution from the sources after all stages $s$ (i.e. after the full time step $\Delta t$). There does not seem to be much difference no matter the choosing. <b>It seems more representative of a "continuous source" to include source at each time substep in this way so we pursue this</b>, so that we do not introduce any unneeded coarseness in our split scheme. Note, adding the source after advecting $x$ was an arbitrary choice, would if we like add it in after each $v_x$ substep. We pursue the former because the electric field calculations that follow an $x$ advection step need to have the most up to date information to calculate the most accurate accelerations (dependent on charge locations, i.e. electrostatic interactions).

Hence, we modify <code>lib.split.scheme</code> by including by incrementing the distribution functions <code>fe</code> and <code>fi</code>, after each <code>x</code> advection and store such a source term in the dictionary <code>sim_params</code> which is read in from the <code>lib.read</code> method and specified by the user in the input deck <code>etc/params.dat</code>, where it is assembled in <code>lib.density.setup</code> (whew!). the form of the source term, e.g. for our problem in this notebook is all zeros except at $x = L$. Hence, adding this array of mostly zeros on top of the densities <code>fe</code> and <code>fi</code> is consistent.

### <font color = "red">lib.split.scheme</font>

In [ ]:
import DECSKS
import time

def scheme(
        fe, fi,
        t,
        x, vx, ax,
        n,
        sim_params
        ):
    """Steps through 1D-1V Vlasov with chosen splitting scheme.

    inputs:
    f -- (ndarray, dim=3) f(t,x,v) with f(0,x,v) = f_0 initialized
    x -- (instance) x, space
    v -- (instance) v, velocity
    t -- (instance) time
    n -- (int) current time step index, t^n
    sim_params -- (dict) simulation parameters

    outputs:
    f -- (ndarray, dim=3) f(n+1,x,v)
    """
    # retrieve sub-dictionary containing splitting coefficients and composition order
    splitting = sim_params['splitting']
    coeff = splitting['order']['coeffs']
    stage = splitting['order']['stages']
    tic = time.time()
    for s in range(len(stage)):
        split_coeff = splitting[coeff[s]][int(stage[s])]
        if coeff[s] == 'a': # advect x
            x.CFL.compute_numbers(x, vx, split_coeff*t.width)
            fe = DECSKS.lib.convect.scheme(
                    fe,
                    n,
                    sim_params,
                    z = x,
                    vz = vx,
                    charge = -1)
            fi = DECSKS.lib.convect.scheme(
                    fi,
                    n,
                    sim_params,
                    z = x,
                    vz = vx,
                    charge = 1)

            fe += sim_params['source_e']
            fi += sim_params['source_i']
            
        elif coeff[s] == 'b': # advect vx
            # calculate electric field at most recent positions of ions and electrons
            #                Ex = DECSKS.lib.fieldsolvers.compute_electric_field_fourier(fe, fi, x, vx, n-1, sim_params)
            Ex = eval(sim_params['compute_electric_field_function_handle'])(fe, fi, x, vx, sim_params)

            # advect electron velocities
            ax.prepointvaluemesh = -Ex
            vx.CFL.compute_numbers(vx, ax, split_coeff*t.width)
            fe = DECSKS.lib.convect.scheme(
                fe,
                n,
                sim_params,
                z = vx,
                vz = ax,
                charge = -1)

            # advect ion velocities
            ax.prepointvaluemesh = 1. / sim_params['mu'] * Ex
            vx.CFL.compute_numbers(vx, ax, split_coeff*t.width)
            fi = DECSKS.lib.convect.scheme(
                fi,
                n,
                sim_params,
                z = vx,
                vz = ax,
                charge = 1)

    toc = time.time()
    print "time step %d of %d completed in %g seconds" % (n,t.N, toc - tic)

    return fe, fi


### TODO

-- include source in etc/params.dat

-- include source terms in lib.density.Setup to create the arrays source_e, source_i

-- include the arrays source_e, source_i in sim_params by inclusion in lib.read return

# References

[1] Kolobov, V.I., Arslanbekov, R.R. Towards adaptive kinetic-fluid simulations of weakly ionized plasmas. Journal of Computational Physics <b>231</b> (2012) 839-869

[2] Christleib, A. et. al. Test Problems for Vlasov-Poisson. Oct. 15, 2014. Slides.